# How to view and update past graph state¶

Once you start checkpointing your graphs, you can easily get or update the state of the agent at any point in time. This permits a few things:
- You can surface a state during an interrupt to a user to let them accept an action.
- You can rewind the graph to reproduce or avoid issues.
- You can modify the state to embed your agent into a larger system, or to let the user better control its actions.

The key methods used for this functionality are:
- get_state: fetch the values from the target config
- update_state: apply the given values to the target state

Note: this requires passing in a checkpointer.

Below is a quick example.



In [1]:
%pip install --quiet -U langgraph langchain_anthropic


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langgraph.graph import MessagesState

# Set up the tool
from langchain_core.tools import tool
from langgraph.prebuilt import ToolExecutor

@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return [
        f"I looked up: {query}. Result: It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    ]


tools = [search]
tool_executor = ToolExecutor(tools)

# Set up the model
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
model = model.bind_tools(tools)

# Define nodes and conditional edges

from langchain_core.messages import ToolMessage

from langgraph.prebuilt import ToolInvocation

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    tool_call = last_message.tool_calls[0]
    action = ToolInvocation(
        tool=tool_call["name"],
        tool_input=tool_call["args"],
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a ToolMessage
    tool_message = ToolMessage(
        content=str(response), name=action.tool, tool_call_id=tool_call["id"]
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [tool_message]}

# Build the graph

from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Set up memory
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)



# Interacting with the Agent
We can now interact with the agent. Let's ask it for the weather in SF.



In [3]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Use the search tool to look up the weather in SF")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

Use the search tool to look up the weather in SF
================================== Ai Message ==================================
Tool Calls:
  search (call_QaaKXnOsCZEB3iSPobymlgpu)
 Call ID: call_QaaKXnOsCZEB3iSPobymlgpu
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: search

["I looked up: weather in San Francisco. Result: It's sunny in San Francisco, but you better look out if you're a Gemini 😈."]
================================== Ai Message ==================================

The weather in San Francisco is currently sunny. However, if you're a Gemini, you might want to be cautious!


# Checking history
Let's browse the history of this thread, from start to finish.



In [4]:
all_states = []
for state in app.get_state_history(config):
    print(state)
    all_states.append(state)
    print("--")

StateSnapshot(values={'messages': []}, next=('__start__',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef33da8-328e-64fc-bfff-cf9a0af78b08'}}, metadata={'source': 'input', 'step': -1, 'writes': {'messages': [HumanMessage(content='Use the search tool to look up the weather in SF')]}}, created_at='2024-06-26T16:38:26.815704+00:00', parent_config=None)
--
StateSnapshot(values={'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='d799a0e7-9a1b-4997-9f63-934cb6f72d78')]}, next=('agent',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef33da8-3294-61c2-8000-1e1504065b4e'}}, metadata={'source': 'loop', 'step': 0, 'writes': None}, created_at='2024-06-26T16:38:26.818082+00:00', parent_config=None)
--
StateSnapshot(values={'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='d799a0e7-9a1b-4997-9f63-934cb6f72d78'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QaaKXnOsCZEB3i

# Replay a state
We can go back to any of these states and restart the agent from there! Let's go back to right before the tool call gets executed.



In [6]:
to_replay = all_states[2]

StateSnapshot(values={'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='d799a0e7-9a1b-4997-9f63-934cb6f72d78'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QaaKXnOsCZEB3iSPobymlgpu', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 54, 'total_tokens': 70}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c1891d4-fd24-408f-882e-6bcdebea049c-0', tool_calls=[{'name': 'search', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_QaaKXnOsCZEB3iSPobymlgpu'}], usage_metadata={'input_tokens': 54, 'output_tokens': 16, 'total_tokens': 70})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef33da8-3984-678e-8001-de497fadedc1'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'agent': {'me

In [7]:
to_replay.values

{'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='d799a0e7-9a1b-4997-9f63-934cb6f72d78'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QaaKXnOsCZEB3iSPobymlgpu', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 54, 'total_tokens': 70}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c1891d4-fd24-408f-882e-6bcdebea049c-0', tool_calls=[{'name': 'search', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_QaaKXnOsCZEB3iSPobymlgpu'}], usage_metadata={'input_tokens': 54, 'output_tokens': 16, 'total_tokens': 70})]}

In [8]:
to_replay.next


('action',)

To replay from this place we just need to pass its config back to the agent. Notice that it just resumes from right where it left all - making a tool call.


In [9]:
for event in app.stream(None, to_replay.config):
    for v in event.values():
        print(v)

{'messages': [ToolMessage(content='["I looked up: weather in San Francisco. Result: It\'s sunny in San Francisco, but you better look out if you\'re a Gemini 😈."]', name='search', tool_call_id='call_QaaKXnOsCZEB3iSPobymlgpu')]}
{'messages': [AIMessage(content="The weather in San Francisco is sunny, but there's a playful warning for Geminis!", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 109, 'total_tokens': 129}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-75040f3e-6e4d-4545-b9c6-c8726e0518b2-0', usage_metadata={'input_tokens': 109, 'output_tokens': 20, 'total_tokens': 129})]}


# Branch off a past state
Using LangGraph's checkpointing, you can do more than just replay past states. You can branch off previous locations to let the agent explore alternate trajectories or to let a user "version control" changes in a workflow.

Let's show how to do this to edit the state at a particular point in time. Let's update the state to change the input to the tool



In [10]:
# Let's now get the last message in the state
# This is the one with the tool calls that we want to update
last_message = to_replay.values['messages'][-1]

# Let's now update the args for that tool call
last_message.tool_calls[0]['args'] = {'query': 'current weather in SF'}

branch_config = app.update_state(
    to_replay.config, {"messages": [last_message]},
)

We can then invoke with this new branch_config to resume running from here with changed state. We can see from the log that the tool was called with different input.



In [11]:
for event in app.stream(None, branch_config):
    for v in event.values():
        print(v)


{'messages': [ToolMessage(content='["I looked up: current weather in SF. Result: It\'s sunny in San Francisco, but you better look out if you\'re a Gemini 😈."]', name='search', tool_call_id='call_QaaKXnOsCZEB3iSPobymlgpu')]}
{'messages': [AIMessage(content="The current weather in San Francisco is sunny. But it seems like there's a playful message for Geminis to look out for!", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 109, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a4e30258-66e6-454a-a530-ab47c24fb667-0', usage_metadata={'input_tokens': 109, 'output_tokens': 28, 'total_tokens': 137})]}


Alternatively, we could update the state to not even call a tool!



In [12]:
from langchain_core.messages import AIMessage

# Let's now get the last message in the state
# This is the one with the tool calls that we want to update
last_message = to_replay.values['messages'][-1]

# Let's now get the ID for the last message, and create a new message with that ID.
new_message = AIMessage(content="its warm!", id=last_message.id)

branch_config = app.update_state(
    to_replay.config, {"messages": [new_message]},
)


In [13]:
branch_state = app.get_state(branch_config)


In [14]:
branch_state.values


{'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='d799a0e7-9a1b-4997-9f63-934cb6f72d78'),
  AIMessage(content='its warm!', id='run-0c1891d4-fd24-408f-882e-6bcdebea049c-0')]}

In [15]:
branch_state.next


()

You can see the snapshot was updated and now correctly reflects that there is no next step.

